# Practice assignment: Advanced ensembling techniques
This assignment is graded by your `submission.json`.

The cell below creates a valid `submission.json` file, fill your answers in there. 

You can press "Submit Assignment" at any time to submit partial progress.

In [16]:
%%file submission.json
{
    "q1": 0,
    "q2": 843300,
    "q3": 1400,
    "q4": 3349.74057,
    "q5": 10329.20768,
    "q6": 8890.23464,
    "q7": 8692.77417,
    "q8": "data_channel_is_bus",
    "q9": 8451.28590,
    "q10": 8421.39852,
    "q11": "self_reference_min_shares",
    "q12": 8422.73009,
    "q13": 8485.01086,
    "q14": 8465.44037,
    "q15": "kw_avg_avg",
    "q16": 8426.978935,
    "q17": 2,
    "q18": 2881.74948,
    "q19": 0.84455,
    "q20": 8439.42046,
    "q21": 8404.95586,
    "q22": 8446.64575
}

Overwriting submission.json


In [3]:
import xgboost
import lightgbm
import catboost

In [5]:
xgboost.__version__

'1.3.3'

In [6]:
lightgbm.__version__

'3.1.1'

In [7]:
catboost.__version__

'0.24.4'

In [12]:
lightgbm.__version__

'3.1.1'

In this programming assignment, you are going to work with a dataset based on the following data:

https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity

_Citation:_

* _K. Fernandes, P. Vinagre and P. Cortez. A Proactive Intelligent Decision Support System for Predicting the Popularity of Online News. Proceedings of the 17th EPIA 2015 - Portuguese Conference on Artificial Intelligence, September, Coimbra, Portugal._

The dataset contains the information about the internet news articles. In this assignment, you are going to predict a number of shares of the news article (target column: `shares`). The information about the features is available through the link above. You are going to construct several machine learning algorithms (XGBoost, LightGBM, CatBoost and Lasso) and blend them into the final ensemble.

In [10]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [3]:
df = pd.read_csv('data.csv')

## 1

**q1:** How many missing values are there in the data? Provide the number of cells in the dataframe that contain NaNs.

In [4]:
# your code here
df.isnull().sum()

n_tokens_title                   0
n_tokens_content                 0
n_unique_tokens                  0
n_non_stop_words                 0
n_non_stop_unique_tokens         0
num_hrefs                        0
num_self_hrefs                   0
num_imgs                         0
num_videos                       0
average_token_length             0
num_keywords                     0
data_channel_is_lifestyle        0
data_channel_is_entertainment    0
data_channel_is_bus              0
data_channel_is_socmed           0
data_channel_is_tech             0
data_channel_is_world            0
kw_min_min                       0
kw_max_min                       0
kw_avg_min                       0
kw_min_max                       0
kw_max_max                       0
kw_avg_max                       0
kw_min_avg                       0
kw_max_avg                       0
kw_avg_avg                       0
self_reference_min_shares        0
self_reference_max_shares        0
self_reference_avg_s

## 2

**q2:** What is the maximum number of shares among all the news articles presented in the data?

In [5]:
# your code here
df['self_reference_max_shares'].max()

843300.0

## 3

**q3:** What is the median number of shares for the articles published on Monday?

In [6]:
# your code here
df[df['weekday_is_monday'] == 1.0]['shares'].median()

1400.0

## 4

First, we separate the target from the dataframe with features (`df` -> `X`, `y`).

Next, let's split the data into train/val/test sets in the ratio 60:20:20. The idea is that we will use train set to train our models, val set to validate them and test set to calculate the final error of the blend. So, test set will be a completely unseen data.

To do this, use a regular `train_test_split` from `sklearn` to split `X` and `y` into train and val/test parts in the ratio 60:40. Then use `train_test_split` again, but to split the obtain val/test part into validation and test in the ratio 50:50. In each `train_test_split` application, use `random_state=13` and other default parameter values.

In the end, you should obtain `X_train`, `X_val`, `X_test` with the following shapes, respectively: (23786, 58), (7929, 58), (7929, 58). The same logic is with `y_train`, `y_val`, `y_test`.

**q4:** What is the mean value of target in the test part (`X_test`)? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [4]:
X = df.drop('shares', axis=1)
y = df['shares']

In [5]:
# your code here
X_train, X_total, y_train, y_total = train_test_split(X, y, test_size = 0.4, random_state = 13)
X_val, X_test, y_val, y_test = train_test_split(X_total, y_total, test_size = 0.5, random_state = 13)

In [9]:
X_train.shape, y_train.shape

((23786, 58), (23786,))

In [10]:
X_val.shape, y_val.shape

((7929, 58), (7929,))

In [11]:
X_test.shape, y_test.shape

((7929, 58), (7929,))

In [12]:
y_test.mean()

3349.740572581662

## 5

Now let's train our first model - XGBoost. A link to the documentation: https://xgboost.readthedocs.io/en/latest/

We will use Scikit-Learn Wrapper interface for XGBoost (and the same logic applies to the following LightGBM and CatBoost models). Here, we work on the regression task - hence we will use `XGBRegressor`. Read about the parameters of `XGBRegressor`: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor

The main list of XGBoost parameters: https://xgboost.readthedocs.io/en/latest/parameter.html# Look through this list so that you understand which parameters are presented in the library.

Take `XGBRegressor` with MSE objective (`objective='reg:squarederror'`), 200 trees (`n_estimators=200`), `learning_rate=0.01`, `max_depth=5`, `random_state=13` and all other default parameter values. Train it on the train set (`fit` function). 

**q5:** Calculate Root Mean Squared Error (RMSE) on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [13]:
# your code here
xgb_boost = XGBRegressor(objective = 'reg:squarederror', n_estimators = 200, learning_rate = 0.01, 
                         max_depth = 5, random_state = 13)
xgb_boost.fit(X_train, y_train)
y_val_predict = xgb_boost.predict(X_val)
MSE_val = mean_squared_error(y_val, y_val_predict, squared = False)

In [14]:
MSE_val

10329.207682578126

## 6

In the task 5, we have decided to build 200 trees in our model. However, it is hard to understand whether it is a good decision - maybe it is too much? Maybe 150 is a better number? Or 100? Or 50 is enough?

During the training process, it is possible to stop constructing the ensemble if we see that the validation error does not decrease anymore. Using the same XGBoost model, call `fit` function (to train it) with `eval_set=[(X_val, y_val)]` (to evaluate the boosting model after building a new tree) and `early_stopping_rounds=50` (and other default parameter values). This `early_stopping_rounds` says that if the validation metric does not increase on 50 consequent iterations, the training stops.

**q6:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [15]:
# your code here
xgb_boost.fit(X_train, y_train, early_stopping_rounds = 50, eval_metric = 'rmse', eval_set = [(X_val, y_val)])
y_val_predict = xgb_boost.predict(X_val)
MSE_val = mean_squared_error(y_val, y_val_predict, squared = False)

[0]	validation_0-rmse:9132.57812
[1]	validation_0-rmse:9118.46875
[2]	validation_0-rmse:9104.95019
[3]	validation_0-rmse:9091.79394
[4]	validation_0-rmse:9079.19824
[5]	validation_0-rmse:9067.49414
[6]	validation_0-rmse:9056.05664
[7]	validation_0-rmse:9045.18359
[8]	validation_0-rmse:9035.23340
[9]	validation_0-rmse:9025.48535
[10]	validation_0-rmse:9016.13965
[11]	validation_0-rmse:9007.63379
[12]	validation_0-rmse:8999.20215
[13]	validation_0-rmse:8991.06543
[14]	validation_0-rmse:8983.98047
[15]	validation_0-rmse:8976.73926
[16]	validation_0-rmse:8970.25977
[17]	validation_0-rmse:8963.87891
[18]	validation_0-rmse:8958.61035
[19]	validation_0-rmse:8953.27148
[20]	validation_0-rmse:8948.50586
[21]	validation_0-rmse:8944.50488
[22]	validation_0-rmse:8938.09570
[23]	validation_0-rmse:8934.25000
[24]	validation_0-rmse:8931.12988
[25]	validation_0-rmse:8925.65137
[26]	validation_0-rmse:8923.33887
[27]	validation_0-rmse:8918.59766
[28]	validation_0-rmse:8916.47266
[29]	validation_0-rmse:8

In [16]:
MSE_val

8890.234644122644

## 7

Notes on parameter tuning: https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html

Here, we tuned some parameters of the algorithm. Take `XGBRegressor` with the following parameters:

* `objective='reg:squarederror'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=4`
* `gamma=1`
* `subsample=0.5`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 6, but with `early_stopping_rounds=500`. 

**q7:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm.

In [ ]:
# your code here
xgb_boost_tune = XGBRegressor(objective='reg:squarederror', n_estimators = 5000, 
                              learning_rate = 0.001, max_depth = 4, gamma = 1, subsample = 0.5,
                             random_state = 13)
xgb_boost_tune.fit(X_train, y_train,early_stopping_rounds = 500, eval_metric = 'rmse', eval_set = [(X_val, y_val)])
y_val_predict = xgb_boost_tune.predict(X_val)
MSE_val = mean_squared_error(y_val, y_val_predict, squared = False)

[0]	validation_0-rmse:9145.78809
[1]	validation_0-rmse:9144.41992
[2]	validation_0-rmse:9143.00977
[3]	validation_0-rmse:9141.59375
[4]	validation_0-rmse:9140.17969
[5]	validation_0-rmse:9138.77637
[6]	validation_0-rmse:9137.16016
[7]	validation_0-rmse:9135.65234
[8]	validation_0-rmse:9134.11328
[9]	validation_0-rmse:9132.71973
[10]	validation_0-rmse:9131.40723
[11]	validation_0-rmse:9129.92773
[12]	validation_0-rmse:9128.53418
[13]	validation_0-rmse:9127.15137
[14]	validation_0-rmse:9125.81445
[15]	validation_0-rmse:9124.47852
[16]	validation_0-rmse:9122.91602
[17]	validation_0-rmse:9121.50098
[18]	validation_0-rmse:9120.15039
[19]	validation_0-rmse:9118.70606
[20]	validation_0-rmse:9117.30273
[21]	validation_0-rmse:9115.88281
[22]	validation_0-rmse:9114.39746
[23]	validation_0-rmse:9112.98047
[24]	validation_0-rmse:9111.45410
[25]	validation_0-rmse:9110.14551
[26]	validation_0-rmse:9108.71973
[27]	validation_0-rmse:9107.29785
[28]	validation_0-rmse:9105.95215
[29]	validation_0-rmse:9

[238]	validation_0-rmse:8879.43848
[239]	validation_0-rmse:8878.55176
[240]	validation_0-rmse:8877.61426
[241]	validation_0-rmse:8876.83398
[242]	validation_0-rmse:8875.86133
[243]	validation_0-rmse:8875.12402
[244]	validation_0-rmse:8874.42090
[245]	validation_0-rmse:8873.50488
[246]	validation_0-rmse:8872.58008
[247]	validation_0-rmse:8871.70996
[248]	validation_0-rmse:8871.08203
[249]	validation_0-rmse:8870.36231
[250]	validation_0-rmse:8869.47461
[251]	validation_0-rmse:8868.80078
[252]	validation_0-rmse:8867.94629
[253]	validation_0-rmse:8867.27832
[254]	validation_0-rmse:8866.61426
[255]	validation_0-rmse:8866.09375
[256]	validation_0-rmse:8865.22168
[257]	validation_0-rmse:8864.37988
[258]	validation_0-rmse:8863.51562
[259]	validation_0-rmse:8862.67090
[260]	validation_0-rmse:8861.73438
[261]	validation_0-rmse:8861.06934
[262]	validation_0-rmse:8860.45508
[263]	validation_0-rmse:8859.58203
[264]	validation_0-rmse:8858.78516
[265]	validation_0-rmse:8858.09180
[266]	validation_0-r

[473]	validation_0-rmse:8745.20019
[474]	validation_0-rmse:8744.85742
[475]	validation_0-rmse:8744.62500
[476]	validation_0-rmse:8744.04688
[477]	validation_0-rmse:8743.80176
[478]	validation_0-rmse:8743.54004
[479]	validation_0-rmse:8742.95117
[480]	validation_0-rmse:8742.78906
[481]	validation_0-rmse:8742.70898
[482]	validation_0-rmse:8742.16211
[483]	validation_0-rmse:8741.57422
[484]	validation_0-rmse:8741.40820
[485]	validation_0-rmse:8741.06836
[486]	validation_0-rmse:8740.76660
[487]	validation_0-rmse:8740.49121
[488]	validation_0-rmse:8740.18457
[489]	validation_0-rmse:8740.04004
[490]	validation_0-rmse:8739.78613
[491]	validation_0-rmse:8739.34277
[492]	validation_0-rmse:8739.29883
[493]	validation_0-rmse:8739.09375
[494]	validation_0-rmse:8738.56641
[495]	validation_0-rmse:8738.39746
[496]	validation_0-rmse:8738.15234
[497]	validation_0-rmse:8737.96484
[498]	validation_0-rmse:8737.49512
[499]	validation_0-rmse:8737.33887
[500]	validation_0-rmse:8736.91992
[501]	validation_0-r

[708]	validation_0-rmse:8697.28320
[709]	validation_0-rmse:8697.33887
[710]	validation_0-rmse:8697.04981
[711]	validation_0-rmse:8696.75781
[712]	validation_0-rmse:8696.81641
[713]	validation_0-rmse:8696.55957
[714]	validation_0-rmse:8696.51758
[715]	validation_0-rmse:8696.70410
[716]	validation_0-rmse:8696.39160
[717]	validation_0-rmse:8696.07324
[718]	validation_0-rmse:8695.96094
[719]	validation_0-rmse:8696.04102
[720]	validation_0-rmse:8696.34277
[721]	validation_0-rmse:8696.39844
[722]	validation_0-rmse:8696.07227
[723]	validation_0-rmse:8696.13574
[724]	validation_0-rmse:8695.91504
[725]	validation_0-rmse:8695.59082
[726]	validation_0-rmse:8695.88965
[727]	validation_0-rmse:8695.94727
[728]	validation_0-rmse:8695.65625
[729]	validation_0-rmse:8695.95508
[730]	validation_0-rmse:8696.06348
[731]	validation_0-rmse:8695.89941
[732]	validation_0-rmse:8695.57519
[733]	validation_0-rmse:8695.69629
[734]	validation_0-rmse:8695.90918
[735]	validation_0-rmse:8695.59766
[736]	validation_0-r

In [18]:
y_val_predict = xgb_boost_tune.predict(X_val)
MSE_val = mean_squared_error(y_val, y_val_predict, squared = False)
MSE_val

8692.774172942118

## 8

Calculate feature importances according to the model, trained in the task 7. 

**q8:** What is the name of the most important feature? Provide it as the answer. Do you understand why it might be important for the model?

Notice that by default, `XGBRegressor` calculates feature importance considering gain (`importance_type` parameter).

In [19]:
# your code here
dic = xgb_boost_tune.get_booster().get_score(importance_type='gain')
dic = sorted(dic.items(), key = lambda x:x[1])
dic

[('weekday_is_thursday', 49148147.84615385),
 ('data_channel_is_tech', 351068494.78479993),
 ('weekday_is_sunday', 488278656.0),
 ('is_weekend', 514930956.97959185),
 ('data_channel_is_socmed', 784552447.3333334),
 ('rate_negative_words', 1039434816.3157895),
 ('data_channel_is_world', 1377030282.6666667),
 ('abs_title_sentiment_polarity', 1852603620.4545455),
 ('data_channel_is_entertainment', 2406554297.5555553),
 ('global_sentiment_polarity', 4362711865.7625),
 ('global_subjectivity', 5168673094.376689),
 ('rate_positive_words', 5196430958.588235),
 ('LDA_01', 6568388151.167883),
 ('max_negative_polarity', 6622301931.284023),
 ('max_positive_polarity', 6734085702.142858),
 ('kw_min_avg', 6995155735.974654),
 ('data_channel_is_lifestyle', 6997688809.846154),
 ('weekday_is_friday', 8187695408.0),
 ('LDA_04', 9254253664.297085),
 ('weekday_is_tuesday', 9571578860.0),
 ('LDA_03', 10572227772.517954),
 ('avg_negative_polarity', 10964136811.789474),
 ('title_sentiment_polarity', 114976066

## 9

Let's move to LightGBM. We will work with `LGBMRegressor`.

LGBMRegressor parameters: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor

The main list of LightGBM parameters: https://lightgbm.readthedocs.io/en/latest/Parameters.html Look through this list so that you understand which parameters are presented in the library.

Take `LGBMRegressor` with the following parameters, similar to the previous `XGBoost` model:

* `objective='regression'`
* `n_estimators=200`
* `learning_rate=0.01`
* `max_depth=5`
* `random_state=13`
* other default parameter values

Train it on the training data with `eval_set=[(X_val, y_val)]`, `eval_metric='rmse'`, `early_stopping_rounds=50` and all other default parameter values. 

**q9:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm and compare it to the speed of XGBoost model.

In [20]:
# your code here
lgb = LGBMRegressor(objective = 'regression', n_estimators = 200, learning_rate = 0.01, 
                    max_depth = 5, random_state = 13)
lgb.fit(X_train, y_train, eval_metric = 'rmse', early_stopping_rounds = 50, eval_set = [(X_val, y_val)])
lgb_val_predict = lgb.predict(X_val)
mse_lgb = mean_squared_error(y_val, lgb_val_predict, squared = False)
mse_lgb

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[1]	valid_0's rmse: 8525.53	valid_0's l2: 7.26847e+07
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 8523.14	valid_0's l2: 7.26439e+07
[3]	valid_0's rmse: 8520.95	valid_0's l2: 7.26065e+07
[4]	valid_0's rmse: 8518.85	valid_0's l2: 7.25708e+07
[5]	valid_0's rmse: 8516.58	valid_0's l2: 7.25321e+07
[6]	valid_0's rmse: 8514.73	valid_0's l2: 7.25007e+07
[7]	valid_0's rmse: 8512.12	valid_0's l2: 7.24563e+07
[8]	valid_0's rmse: 8510.48	valid_0's l2: 7.24282e+07
[9]	valid_0's rmse: 8508.06	valid_0's l2: 7.23871e+07
[10]	valid_0's rmse: 8506.35	valid_0's l2: 7.2358e+07
[11]	valid_0's rmse: 8504.05	valid_0's l2: 7.23188e+07
[12]	valid_0's rmse: 8502.03	valid_0's l2: 7.22844e+07
[13]	valid_0's rmse: 8499.83	valid_0's l2: 7.22471e+07
[14]	valid_0's rmse: 8497.97	valid_0's l2: 7.22156e+07
[15]	valid_0's rmse: 8496.59	valid_0's l2: 7.2192

8451.28590030947

## 10

Notes on parameter tuning: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

Here, we tuned some parameters of the algorithm. Take `LGBMRegressor` with the following parameters:

* `objective='regression'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=3`
* `lambda_l2=1.0`
* `boosting_type='goss'`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 9, but with `early_stopping_rounds=500`. 

**q10:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:

# your code here
lgb_tune = LGBMRegressor(objective = 'regression', n_estimators = 5000, learning_rate = 0.001,
                        max_depth = 3, lambda_l2 = 1.0, boosting_type = 'goss', random_state = 13)
lgb_tune.fit(X_train, y_train, eval_metric = 'rmse', early_stopping_rounds = 500, eval_set = [(X_val, y_val)])

[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[1]	valid_0's rmse: 8527.68	valid_0's l2: 7.27214e+07
Training until validation scores don't improve for 500 rounds
[2]	valid_0's rmse: 8527.44	valid_0's l2: 7.27172e+07
[3]	valid_0's rmse: 8527.2	valid_0's l2: 7.27131e+07
[4]	valid_0's rmse: 8526.96	valid_0's l2: 7.2709e+07
[5]	valid_0's rmse: 8526.71	valid_0's l2: 7.27049e+07
[6]	valid_0's rmse: 8526.47	valid_0's l2: 7.27008e+07
[7]	valid_0's rmse: 8526.23	valid_0's l2: 7.26967e+07
[8]	valid_0's rmse: 8525.98	valid_0's l2: 7.26924e+07
[9]	valid_0's rmse: 8525.74	valid_0's l2: 7.26883e+07
[10]	valid_0's rmse: 8525.49	valid_0's l2: 7.26841e+07
[11]	valid_0's rmse: 8525.26	valid_0's l2: 7.268e+07
[12]	valid_0's rmse: 8525.01	valid_0's l2: 7.26758e+07
[13]	valid_0's rmse: 8524.77	valid_0's l2: 7.26717e+07
[14]	valid_0's rmse: 8524.52	valid_0's l2: 7.26675e+07
[15]	valid_0's rmse: 8524.29	valid_0's l2: 7.26635e+07
[16]	valid_0's rmse: 8

[147]	valid_0's rmse: 8495.57	valid_0's l2: 7.21748e+07
[148]	valid_0's rmse: 8495.37	valid_0's l2: 7.21713e+07
[149]	valid_0's rmse: 8495.19	valid_0's l2: 7.21683e+07
[150]	valid_0's rmse: 8494.99	valid_0's l2: 7.21648e+07
[151]	valid_0's rmse: 8494.82	valid_0's l2: 7.21619e+07
[152]	valid_0's rmse: 8494.61	valid_0's l2: 7.21584e+07
[153]	valid_0's rmse: 8494.4	valid_0's l2: 7.21549e+07
[154]	valid_0's rmse: 8494.23	valid_0's l2: 7.2152e+07
[155]	valid_0's rmse: 8494.03	valid_0's l2: 7.21485e+07
[156]	valid_0's rmse: 8493.86	valid_0's l2: 7.21456e+07
[157]	valid_0's rmse: 8493.65	valid_0's l2: 7.21422e+07
[158]	valid_0's rmse: 8493.46	valid_0's l2: 7.21388e+07
[159]	valid_0's rmse: 8493.28	valid_0's l2: 7.21358e+07
[160]	valid_0's rmse: 8493.08	valid_0's l2: 7.21324e+07
[161]	valid_0's rmse: 8492.91	valid_0's l2: 7.21295e+07
[162]	valid_0's rmse: 8492.71	valid_0's l2: 7.21261e+07
[163]	valid_0's rmse: 8492.54	valid_0's l2: 7.21233e+07
[164]	valid_0's rmse: 8492.34	valid_0's l2: 7.2119

[297]	valid_0's rmse: 8472.67	valid_0's l2: 7.17861e+07
[298]	valid_0's rmse: 8472.51	valid_0's l2: 7.17835e+07
[299]	valid_0's rmse: 8472.4	valid_0's l2: 7.17816e+07
[300]	valid_0's rmse: 8472.28	valid_0's l2: 7.17795e+07
[301]	valid_0's rmse: 8472.17	valid_0's l2: 7.17777e+07
[302]	valid_0's rmse: 8472.07	valid_0's l2: 7.17759e+07
[303]	valid_0's rmse: 8471.91	valid_0's l2: 7.17733e+07
[304]	valid_0's rmse: 8471.8	valid_0's l2: 7.17714e+07
[305]	valid_0's rmse: 8471.69	valid_0's l2: 7.17695e+07
[306]	valid_0's rmse: 8471.58	valid_0's l2: 7.17676e+07
[307]	valid_0's rmse: 8471.48	valid_0's l2: 7.17659e+07
[308]	valid_0's rmse: 8471.32	valid_0's l2: 7.17633e+07
[309]	valid_0's rmse: 8471.21	valid_0's l2: 7.17615e+07
[310]	valid_0's rmse: 8471.1	valid_0's l2: 7.17595e+07
[311]	valid_0's rmse: 8470.99	valid_0's l2: 7.17577e+07
[312]	valid_0's rmse: 8470.89	valid_0's l2: 7.1756e+07
[313]	valid_0's rmse: 8470.74	valid_0's l2: 7.17535e+07
[314]	valid_0's rmse: 8470.63	valid_0's l2: 7.17515e

[444]	valid_0's rmse: 8458.12	valid_0's l2: 7.15399e+07
[445]	valid_0's rmse: 8458.04	valid_0's l2: 7.15384e+07
[446]	valid_0's rmse: 8457.96	valid_0's l2: 7.15371e+07
[447]	valid_0's rmse: 8457.88	valid_0's l2: 7.15357e+07
[448]	valid_0's rmse: 8457.79	valid_0's l2: 7.15342e+07
[449]	valid_0's rmse: 8457.72	valid_0's l2: 7.1533e+07
[450]	valid_0's rmse: 8457.64	valid_0's l2: 7.15316e+07
[451]	valid_0's rmse: 8457.55	valid_0's l2: 7.15302e+07
[452]	valid_0's rmse: 8457.48	valid_0's l2: 7.15289e+07
[453]	valid_0's rmse: 8457.4	valid_0's l2: 7.15276e+07
[454]	valid_0's rmse: 8457.32	valid_0's l2: 7.15263e+07
[455]	valid_0's rmse: 8457.24	valid_0's l2: 7.15249e+07
[456]	valid_0's rmse: 8457.16	valid_0's l2: 7.15236e+07
[457]	valid_0's rmse: 8457.09	valid_0's l2: 7.15223e+07
[458]	valid_0's rmse: 8457	valid_0's l2: 7.15209e+07
[459]	valid_0's rmse: 8456.93	valid_0's l2: 7.15197e+07
[460]	valid_0's rmse: 8456.85	valid_0's l2: 7.15184e+07
[461]	valid_0's rmse: 8456.77	valid_0's l2: 7.1517e+0

[591]	valid_0's rmse: 8447.61	valid_0's l2: 7.13621e+07
[592]	valid_0's rmse: 8447.52	valid_0's l2: 7.13605e+07
[593]	valid_0's rmse: 8447.45	valid_0's l2: 7.13594e+07
[594]	valid_0's rmse: 8447.39	valid_0's l2: 7.13583e+07
[595]	valid_0's rmse: 8447.29	valid_0's l2: 7.13568e+07
[596]	valid_0's rmse: 8447.27	valid_0's l2: 7.13564e+07
[597]	valid_0's rmse: 8447.21	valid_0's l2: 7.13553e+07
[598]	valid_0's rmse: 8447.16	valid_0's l2: 7.13545e+07
[599]	valid_0's rmse: 8447.07	valid_0's l2: 7.13529e+07
[600]	valid_0's rmse: 8446.97	valid_0's l2: 7.13514e+07
[601]	valid_0's rmse: 8446.93	valid_0's l2: 7.13507e+07
[602]	valid_0's rmse: 8446.89	valid_0's l2: 7.135e+07
[603]	valid_0's rmse: 8446.8	valid_0's l2: 7.13484e+07
[604]	valid_0's rmse: 8446.75	valid_0's l2: 7.13476e+07
[605]	valid_0's rmse: 8446.68	valid_0's l2: 7.13464e+07
[606]	valid_0's rmse: 8446.65	valid_0's l2: 7.1346e+07
[607]	valid_0's rmse: 8446.56	valid_0's l2: 7.13444e+07
[608]	valid_0's rmse: 8446.49	valid_0's l2: 7.13432e

...

[1322]	valid_0's rmse: 8423.85	valid_0's l2: 7.09613e+07
[1323]	valid_0's rmse: 8423.88	valid_0's l2: 7.09617e+07
[1324]	valid_0's rmse: 8423.82	valid_0's l2: 7.09607e+07
[1325]	valid_0's rmse: 8423.74	valid_0's l2: 7.09594e+07
[1326]	valid_0's rmse: 8423.73	valid_0's l2: 7.09593e+07
[1327]	valid_0's rmse: 8423.71	valid_0's l2: 7.09589e+07
[1328]	valid_0's rmse: 8423.71	valid_0's l2: 7.09588e+07
[1329]	valid_0's rmse: 8423.71	valid_0's l2: 7.09589e+07
[1330]	valid_0's rmse: 8423.71	valid_0's l2: 7.09588e+07
[1331]	valid_0's rmse: 8423.71	valid_0's l2: 7.0959e+07
[1332]	valid_0's rmse: 8423.7	valid_0's l2: 7.09586e+07
[1333]	valid_0's rmse: 8423.67	valid_0's l2: 7.09581e+07
[1334]	valid_0's rmse: 8423.66	valid_0's l2: 7.0958e+07
[1335]	valid_0's rmse: 8423.6	valid_0's l2: 7.0957e+07
[1336]	valid_0's rmse: 8423.59	valid_0's l2: 7.09569e+07
[1337]	valid_0's rmse: 8423.55	valid_0's l2: 7.09562e+07
[1338]	valid_0's rmse: 8423.46	valid_0's l2: 7.09546e+07
[1339]	valid_0's rmse: 8423.45	valid

[1467]	valid_0's rmse: 8422.23	valid_0's l2: 7.0934e+07
[1468]	valid_0's rmse: 8422.23	valid_0's l2: 7.09339e+07
[1469]	valid_0's rmse: 8422.22	valid_0's l2: 7.09338e+07
[1470]	valid_0's rmse: 8422.2	valid_0's l2: 7.09334e+07
[1471]	valid_0's rmse: 8422.2	valid_0's l2: 7.09334e+07
[1472]	valid_0's rmse: 8422.16	valid_0's l2: 7.09328e+07
[1473]	valid_0's rmse: 8422.16	valid_0's l2: 7.09327e+07
[1474]	valid_0's rmse: 8422.15	valid_0's l2: 7.09326e+07
[1475]	valid_0's rmse: 8422.15	valid_0's l2: 7.09326e+07
[1476]	valid_0's rmse: 8422.14	valid_0's l2: 7.09325e+07
[1477]	valid_0's rmse: 8422.14	valid_0's l2: 7.09324e+07
[1478]	valid_0's rmse: 8422.13	valid_0's l2: 7.09323e+07
[1479]	valid_0's rmse: 8422.11	valid_0's l2: 7.09319e+07
[1480]	valid_0's rmse: 8422.06	valid_0's l2: 7.09311e+07
[1481]	valid_0's rmse: 8422.04	valid_0's l2: 7.09308e+07
[1482]	valid_0's rmse: 8422.06	valid_0's l2: 7.09311e+07
[1483]	valid_0's rmse: 8422.07	valid_0's l2: 7.09312e+07
[1484]	valid_0's rmse: 8422.06	val

[1612]	valid_0's rmse: 8421.91	valid_0's l2: 7.09286e+07
[1613]	valid_0's rmse: 8421.88	valid_0's l2: 7.09281e+07
[1614]	valid_0's rmse: 8421.84	valid_0's l2: 7.09275e+07
[1615]	valid_0's rmse: 8421.84	valid_0's l2: 7.09273e+07
[1616]	valid_0's rmse: 8421.78	valid_0's l2: 7.09265e+07
[1617]	valid_0's rmse: 8421.77	valid_0's l2: 7.09262e+07
[1618]	valid_0's rmse: 8421.77	valid_0's l2: 7.09263e+07
[1619]	valid_0's rmse: 8421.78	valid_0's l2: 7.09265e+07
[1620]	valid_0's rmse: 8421.8	valid_0's l2: 7.09266e+07
[1621]	valid_0's rmse: 8421.81	valid_0's l2: 7.09269e+07
[1622]	valid_0's rmse: 8421.8	valid_0's l2: 7.09267e+07
[1623]	valid_0's rmse: 8421.81	valid_0's l2: 7.09269e+07
[1624]	valid_0's rmse: 8421.8	valid_0's l2: 7.09268e+07
[1625]	valid_0's rmse: 8421.81	valid_0's l2: 7.09268e+07
[1626]	valid_0's rmse: 8421.78	valid_0's l2: 7.09264e+07
[1627]	valid_0's rmse: 8421.79	valid_0's l2: 7.09265e+07
[1628]	valid_0's rmse: 8421.77	valid_0's l2: 7.09263e+07
[1629]	valid_0's rmse: 8421.8	vali

[1757]	valid_0's rmse: 8421.88	valid_0's l2: 7.0928e+07
[1758]	valid_0's rmse: 8421.86	valid_0's l2: 7.09278e+07
[1759]	valid_0's rmse: 8421.89	valid_0's l2: 7.09282e+07
[1760]	valid_0's rmse: 8421.89	valid_0's l2: 7.09282e+07
[1761]	valid_0's rmse: 8421.94	valid_0's l2: 7.0929e+07
[1762]	valid_0's rmse: 8421.9	valid_0's l2: 7.09284e+07
[1763]	valid_0's rmse: 8421.91	valid_0's l2: 7.09285e+07
[1764]	valid_0's rmse: 8421.89	valid_0's l2: 7.09283e+07
[1765]	valid_0's rmse: 8421.94	valid_0's l2: 7.09291e+07
[1766]	valid_0's rmse: 8421.93	valid_0's l2: 7.0929e+07
[1767]	valid_0's rmse: 8421.87	valid_0's l2: 7.09278e+07
[1768]	valid_0's rmse: 8421.87	valid_0's l2: 7.09279e+07
[1769]	valid_0's rmse: 8421.88	valid_0's l2: 7.0928e+07
[1770]	valid_0's rmse: 8421.89	valid_0's l2: 7.09282e+07
[1771]	valid_0's rmse: 8421.92	valid_0's l2: 7.09287e+07
[1772]	valid_0's rmse: 8421.92	valid_0's l2: 7.09287e+07
[1773]	valid_0's rmse: 8421.93	valid_0's l2: 7.09289e+07
[1774]	valid_0's rmse: 8421.95	valid

[1902]	valid_0's rmse: 8423.48	valid_0's l2: 7.0955e+07
[1903]	valid_0's rmse: 8423.46	valid_0's l2: 7.09547e+07
[1904]	valid_0's rmse: 8423.49	valid_0's l2: 7.09553e+07
[1905]	valid_0's rmse: 8423.48	valid_0's l2: 7.09551e+07
[1906]	valid_0's rmse: 8423.45	valid_0's l2: 7.09545e+07
[1907]	valid_0's rmse: 8423.5	valid_0's l2: 7.09553e+07
[1908]	valid_0's rmse: 8423.55	valid_0's l2: 7.09562e+07
[1909]	valid_0's rmse: 8423.53	valid_0's l2: 7.09559e+07
[1910]	valid_0's rmse: 8423.56	valid_0's l2: 7.09564e+07
[1911]	valid_0's rmse: 8423.58	valid_0's l2: 7.09567e+07
[1912]	valid_0's rmse: 8423.5	valid_0's l2: 7.09553e+07
[1913]	valid_0's rmse: 8423.5	valid_0's l2: 7.09554e+07
[1914]	valid_0's rmse: 8423.52	valid_0's l2: 7.09557e+07
[1915]	valid_0's rmse: 8423.55	valid_0's l2: 7.09561e+07
[1916]	valid_0's rmse: 8423.55	valid_0's l2: 7.09563e+07
[1917]	valid_0's rmse: 8423.56	valid_0's l2: 7.09564e+07
[1918]	valid_0's rmse: 8423.6	valid_0's l2: 7.09571e+07
[1919]	valid_0's rmse: 8423.61	valid

In [ ]:
y_val_tune = lgb_tune.predict(X_val)
mse_lgb_tune = mean_squared_error(y_val, y_val_tune, squared = False)
mse_lgb_tune

## 11

Calculate feature importances according to the model, trained in the task 10. 

**q11:** What is the name of the most important feature? Provide it as the answer. 

Do you understand why it might be important for the model?

Notice that by default, `LGBMRegressor` calculates feature importance considering number of times the feature is used in the model (`importance_type` parameter).

In [ ]:
pd.DataFrame({'Value':lgb_tune.feature_importances_,'Feature':lgb_tune.feature_name_}).sort_values(by="Value",ascending=False)

## 12

Since some features are not important for the model, we can drop them in order to try to construct a better model which does not consider them at all.

Obtain new train and validation sets without the features with LightGBM importance less than 10 (the importances were computed in the task 11). Train the same model as in the task 10 on the new train set in the same manner. 

**q12:** Calculate RMSE on the new validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice that the new versions of train and validation sets are used only in this task and in blending.

In [ ]:
lgb_features1 = pd.DataFrame({'Value':lgb_tune.feature_importances_,'Feature':lgb_tune.feature_name_})

In [ ]:
selected_features1 = lgb_features1[lgb_features1['Value']>=10].Feature.tolist()
selected_features1

In [ ]:
selected_features == selected_features1

In [ ]:
# your code here
lgb_features = lgb_tune.feature_importances_
lgb_feature_names = lgb_tune.feature_name_

In [ ]:
selected_features = [X.columns[index] for index in range(len(X.columns)) if lgb_features[index] >= 10]
selected_features

In [ ]:
X_train_new = X_train[selected_features]
X_val_new = X_val[selected_features]

In [ ]:
lgb_new = LGBMRegressor(objective = 'regression', n_estimators = 5000, learning_rate = 0.001,
                        max_depth = 3, lambda_l2 = 1.0, boosting_type = 'goss', random_state = 13)
lgb_new.fit(X_train_new, y_train, eval_metric = 'rmse', early_stopping_rounds = 500, eval_set = [(X_val_new, y_val)])

In [ ]:
y_val_new_predict = lgb_new.predict(X_val_new)
mse_new = mean_squared_error(y_val_new_predict, y_val, squared = False)
mse_new

## 13

Let's move to CatBoost. We will work with `CatBoostRegressor`.

Info about `CatBoostRegressor`: https://catboost.ai/docs/concepts/python-reference_catboostregressor.html

CatBoost parameters: https://catboost.ai/docs/concepts/python-reference_parameters-list.html Look through this list so that you understand which parameters are presented in the library.

Take `CatBoostRegressor` with the following parameters, similar to the previous models:

* `loss_function='RMSE'`
* `iterations=200`
* `learning_rate=0.01`
* `max_depth=5`
* `random_state=13`
* other default parameter values

Train it on the training data with `eval_set=[(X_val, y_val)]`, `early_stopping_rounds=50` and all other default parameter values. 

**q13:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm and compare it to the speed of XGBoost and LightGBM models.

In [ ]:
# your code here
cat_boost = CatBoostRegressor(loss_function = 'RMSE',iterations = 200, learning_rate = 0.01, 
                              max_depth = 5, random_state = 13)
cat_boost.fit(X_train, y_train, early_stopping_rounds = 50,eval_set = [(X_val, y_val)])

In [ ]:
y_val_predict = cat_boost.predict(X_val)
mse_cat_boost = mean_squared_error(y_val, y_val_predict, squared = False)
mse_cat_boost

## 14

Notes on parameter tuning: https://catboost.ai/docs/concepts/parameter-tuning.html

Here, we tuned some parameters of the algorithm. Take `CatBoostRegressor` with the following parameters:

* `loss_function='RMSE'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=9`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 13, but with `early_stopping_rounds=500`. 

**q14:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here
cat_boost_tune = CatBoostRegressor(loss_function = 'RMSE', n_estimators = 5000, learning_rate = 0.001,
                                  max_depth = 9, random_state = 13)
cat_boost_tune.fit(X_train, y_train, early_stopping_rounds = 500, eval_set = [(X_val, y_val)])

In [ ]:
y_val_tune = cat_boost_tune.predict(X_val)
mse_boost_tune = mean_squared_error(y_val, y_val_tune, squared = False)
mse_boost_tune

## 15

Calculate feature importances according to the model, trained in the task 14. 

**q15:** What is the name of the most important feature? Provide it as the answer. 

Do you understand why it might be important for the model?

Notice that in case of regression, `CatBoostRegressor` calculates feature importance considering PredictionValuesChange: https://catboost.ai/docs/concepts/fstr.html#fstr__regular-feature-importance

In [ ]:
# your code here
cat_boost_feature_importances = cat_boost_tune.get_feature_importance()
cat_boost_indices = cat_boost_feature_importances.argsort()
pd.DataFrame({'Value':cat_boost_tune.feature_importances_,'Feature':cat_boost_tune.feature_names_}).sort_values(by="Value",ascending=False)

In [ ]:
cat_boost_feature_importances

## 16

Finally, take a `Lasso` model from `sklearn` with `alpha=10.0`, `random_state=13` and all other default parameter values. Train it on the train set. 

**q16:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here
lasso = Lasso(alpha = 10.0, random_state = 13)
lasso.fit(X_train, y_train)

y_val_predict = lasso.predict(X_val)
mse_y_val_lasso = mean_squared_error(y_val, y_val_predict, squared = False)
mse_y_val_lasso

## 17

Compare the results on the validation set of the trained models:

* XGBoost (task 7)
* LightGBM (task 12)
* CatBoost (task 14)
* Lasso (task 16)

**q17:** Which model has the best RMSE value on the validation set? For the answer, provide the following:

* 1 (if XGBoost was the best)
* 2 (if LightGBM was the best)
* 3 (if CatBoost was the best)
* 4 (if Lasso was the best)

In [ ]:
#task 7
#8692.77417
#task 12
#8422.73008
#task 14
#8465.44037
#task 16
#8426.97894

## 18

Finally, let's move to blending the models that we obtained. First, calculate the predictions for the trained models on the validation set. Remember that LightGBM model used slightly different set of columns in the data.

After getting the predictions for the validation set, concatenate them into a single dataframe `X_val_blend`. The dataframe should look like this:

||xgb|lgb|cb|lasso|
|-|-|-|-|-|
|0|2298.947754|3728.088336|3680.924182|4270.039931|
|1|3208.189209|5243.744431|4487.549790|6755.853939|
|...|...|...|...|...|

Here, `xgb` column represents XGBoost predictions, `lgb` - LightGBM predictions, `cb` - CatBoost predictions, `lasso` - lasso predictions.

**q18:** For the answer, calculate the mean value of all model predictions in the last row of this column (`X_val_blend.iloc[-1]`). Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here
xgb_val_predict = xgb_boost_tune.predict(X_val)
lgb_val_predict = lgb_new.predict(X_val_new)
cb_val_predict = cat_boost_tune.predict(X_val)
lasso_val_predict = lasso.predict(X_val)

X_val_blend = pd.DataFrame({'xgb': xgb_val_predict, 'lgb': lgb_val_predict, 
                            'cb': cb_val_predict, 'lasso': lasso_val_predict})

In [ ]:
X_val_blend

In [ ]:
X_val_blend.iloc[-1].mean()

## 19

Obtain a matrix of pairwise Pearson Correlation Coefficient (PCC) values for the column of the dataframe `X_val_blend`. Find a pair of model predictions with the highest PCC value (don't consider 1.0 values of correlations with themselves). 

**q19:** What is this value equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here
X_val_blend.corr()

## 20

Blend models into the ensemble with the weights 0.25, 0.25, 0.25 and 0.25 (just mean value of the predictions). 

**q20:** Calculate RMSE of such ensemble on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Compare it with RMSE of each model and think whether this is a good ensemble.

In [ ]:
# your code here
from sklearn.linear_model import LogisticRegression

In [ ]:
y_ensemble_predict = X_val_blend.mean(axis = 0)
mse_ensemble = mean_squared_error(y_ensemble_predict, y_val, squared = False)

mse_ensemble

## 21

Tune the weights of the ensemble. Run each model weight through `np.linspace(0, 1, 101)`, so that all possible values of each weight will be [0.0, 0.01, 0.02, ..., 0.99, 1.0]. Skip each combinations of weights, if their sum is not equal to 1.0. If the sum of the weights in the combination is equal to 1.0, though, get ensemble prediction on the validation set using these weights and calculate RMSE value.

In the end, select a combination of weights with the best RMSE value - these are the best weights for the ensemble. 

**q21:** What is their corresponding RMSE value equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Compare RMSE value of the ensemble with RMSE values of the models in it. Is the ensemble better?

_Hint. You probably want to save RMSE with the corresponding weights for each valid combination into some array. Also this weight tuning might be implemented as quadriple nested loop, or you may think about other ways of implementing it. You can track tuning progress using `tqdm` module._

In [ ]:
# your code here

## 22

Using the best weights obtained in the task 21, run the best ensemble on the test set. To do this, obtain model predictions on the test set (you can write them to the similar table to the one for the validation set in the task 18). Remember that LightGBM model uses slightly different set of columns.

**q22:** Calculate RMSE of the final ensemble on the test set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here